# 示例：利用PCA对半导体制造数据降维

In [1]:
import numpy as np

In [2]:
# 加载数据
def loadDataSet(fileName, delim='\t'):
    fr = open(fileName)
    stringArr = [line.strip().split(delim) for line in fr.readlines()]
    datArr = [list(map(float,line)) for line in stringArr]
    return np.mat(datArr)

# pca
# dataMat: 用于进行PCA操作的数据集，topNfeat：应用的N个特征
def pca(dataMat, topNfeat=9999999):
    # 按列得到均值，即求各个特征的均值
    meanVals = dataMat.mean(0)
    # 计算并减去原始数据集的平均值
    meanRemoved = dataMat - meanVals
    # 计算协方差，若rowvar=0，说明传入的数据一行代表一个样本，
    # 若非0，说明传入的数据一列代表一个样本。
    # 因为meanRemoved每一行代表一个样本，所以将rowvar设置为0
    covMat = np.cov(meanRemoved, rowvar=0)
    # 求特征值，返回特征值和特征向量
    eigVals,eigVects = np.linalg.eig(np.mat(covMat))
    # 对特征值进行排序（从小到大）
    eigValInd = np.argsort(eigVals)
    # 最大的topNfeat个特征值的下标
    eigValInd = eigValInd[:-(topNfeat+1):-1]
    # 最大的topNfeat个特征值对应的特征向量
    redEigVects = eigVects[:,eigValInd]
    # 将数据转换到新空间（得到低维特征空间的数据集）
    lowDDataMat = meanRemoved * redEigVects
    # 重构数据
    reconMat = (lowDDataMat * redEigVects.T) + meanVals
    return lowDDataMat, reconMat

In [3]:
#将NaN替换成平均值的函数
def replaceNanWithMean(): 
    datMat = loadDataSet('secom.data', ' ')
    # 获得特征的数目
    numFeat = np.shape(datMat)[1]
    for i in range(numFeat):
        # 计算非NaN值的平均值。
        meanVal = np.mean(datMat[np.nonzero(~np.isnan(datMat[:,i].A))[0],i]) #values that are not NaN (a number)
        # 将所有NaN替换为该平均值
        datMat[np.nonzero(np.isnan(datMat[:,i].A))[0],i] = meanVal  #set NaN values to mean
    return datMat

In [4]:
dataMat = replaceNanWithMean()

借用pca()函数中的代码，了解中间结果而非最后输出结果

In [5]:
meanVals = np.mean(dataMat, axis=0)
meanRemoved = dataMat - meanVals
covMat = np.cov(meanRemoved, rowvar=0)
eigVals, eigVects = np.linalg.eig(np.mat(covMat))

In [6]:
eigVals

array([ 5.34151979e+07,  2.17466719e+07,  8.24837662e+06,  2.07388086e+06,
        1.31540439e+06,  4.67693557e+05,  2.90863555e+05,  2.83668601e+05,
        2.37155830e+05,  2.08513836e+05,  1.96098849e+05,  1.86856549e+05,
        1.52422354e+05,  1.13215032e+05,  1.08493848e+05,  1.02849533e+05,
        1.00166164e+05,  8.33473762e+04,  8.15850591e+04,  7.76560524e+04,
        6.66060410e+04,  6.52620058e+04,  5.96776503e+04,  5.16269933e+04,
        5.03324580e+04,  4.54661746e+04,  4.41914029e+04,  4.15532551e+04,
        3.55294040e+04,  3.31436743e+04,  2.67385181e+04,  1.47123429e+04,
        1.44089194e+04,  1.09321187e+04,  1.04841308e+04,  9.48876548e+03,
        8.34665462e+03,  7.22765535e+03,  5.34196392e+03,  4.95614671e+03,
        4.23060022e+03,  4.10673182e+03,  3.41199406e+03,  3.24193522e+03,
        2.74523635e+03,  2.35027999e+03,  2.16835314e+03,  1.86414157e+03,
        1.76741826e+03,  1.70492093e+03,  1.66199683e+03,  1.53948465e+03,
        1.33096008e+03,  

后面有很多值都是0，其中有超过20%的特征值都是0，也就是说，他们可以通过其他特征来表示，而本身并没有提供额外的信息。